In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!mkdir -p ~/.kaggle
!echo '{"username":"nolaurence","key":"581f65fa2f236667ac3989e5c525fbbb"}' > ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d retailrocket/ecommerce-dataset

import zipfile
zipFile = zipfile.ZipFile('/content/ecommerce-dataset.zip')
for file in zipFile.namelist():
    zipFile.extract(file, '/content')
zipFile.close()

ecommerce-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [0]:
import numpy as np
import pandas as pd
import tensorflow as tf
import argparse
np.random.seed(1)

In [0]:
# 构建商品图
# data needed：用户id，与用户交互过的商品，数量，商品。
# data directory:
# /content/category_tree.csv
# /content/events.csv
# /content/item_properties_part1.csv
# /content/item_properties_part2.csv

In [0]:
# x = []
# for u in users:
#   x.append(str(u))

# 构建每个用户交互过的商品的数据集n
# sortedbh = behaviordata.sort_values(by='visitorid')

In [0]:
# 创建一个类似于上一个cell，item对于userid的字典m

# np.save('/content/item2graph.npy', m)
# print('saved')

In [0]:
# 构建商品图V
# 计划采用一个列表来表示 n * 2大小 每一行第一个表示边
# 还有一种就是传统的邻接矩阵来表示，但是矩阵构造的过程涉及到两个循环的嵌套。。
# 应当极力避免循环嵌套。。
# 2019/11/09 15：00尝试邻接矩阵方法构建
# try failed
# 2019/11/09 20:00 尝试使用邻接表方法  failed
# 2019/11/09 21:04 尝试设计新的方法，主要失败原因：内存溢出
# 2019/11/09 22:00 新方法有效 设计成功
# 2019/11/10 14:50 使用numpy手段存储跑出来的数据 成功

# graph = np.array(graph)
# np.savetxt('/content/drive/My Drive/Colab Notebooks/graph.csv', graph, delimiter=',')
# print('saved')
# 无法存储数据
# np.save('/content/itemGraph.npy', graph)
# print('saved')

In [0]:
# 2019/11/10 尝试实现itemgraph的去重
# 2019/11/10 17：09完成itemgraph的去重，并保存为了itemgraphv2.npy文件
# 数量条目为32,646,728
# 2019/11/10 17：10开始实现adamic算法计算edge weight
# 2019/11/10 20:22 发现itemgraphv2有问题
# 2019/11/16 insert失败的主要原因是adamvalues与graph图的数据类型不同
item2user = np.load('/content/drive/My Drive/Colab Notebooks/data/item2user.npy', allow_pickle=True).item()
user2item = np.load('/content/drive/My Drive/Colab Notebooks/data/user2item.npy', allow_pickle=True).item()
itemgraph = np.load('/content/drive/My Drive/Colab Notebooks/data/newgraph.npy')
adamvalues = np.load('/content/drive/My Drive/Colab Notebooks/data/adamValues.npy')

In [0]:
# 可用的商品图去重方法
# graph = np.load('/content/drive/My Drive/Colab Notebooks/data/itemGraph.npy') 
# print(itemgraph.shape)
# for i in range(itemgraph.shape[0]):
#     if i % 100000 == 0:
#         print('\r', '{:.2f} % '.format(i / itemgraph.shape[0] * 100), end='')
#     if itemgraph[i][0] > itemgraph[i][1]:
#         x = itemgraph[i][1]
#         itemgraph[i][1] = itemgraph[i][0]
#         itemgraph[i][0] = x
# newgraph = np.array(list(set([tuple(t) for t in itemgraph])))
# newgraph.shape
# np.save('/content/newgraph.npy', newgraph)

In [0]:
# 定义一些超参数
parser = argparse.ArgumentParser()

parser.add_argument('--dataset', type=str, default='kaggle', help='set aside')
parser.add_argument('--pooling', type=str, default='adamic', help='which pooling method to use')
parser.add_argument('--n_epochs', type=int, default=10, help=' the number of epochs')
parser.add_argument('--sample_size', type=int, default=3, help='the number of kidnodes of every node')
parser.add_argument('--dim', type=int, default=32, help='number of embedding vector, choose in [8, 16, 32]')
parser.add_argument('--k', type=int, default=3, help='the depth of tree')
parser.add_argument('--batch_size', type=int, default=128, help='batch size')
parser.add_argument('--l2_weight', type=float, default=1e-6, help='weight of l2 regularization in 1e-6~1')
parser.add_argument('--lr', type=float, default=1e-2, help='learning rate')

In [0]:
def takeSecond(elem):
    return elem[1]

# 2019/11/16 23：13完成子图规则化
def construct_graph(itemgraph, adamvalues, n_sample):
    print('constructing itemgraph ...')
    graph = dict()
    for i in range(itemgraph.shape[0]):
        if i % 10000 == 0:
            print('\r', '{:.2f} %'.format(i / itemgraph.shape[0] * 100), end='')
        first = itemgraph[i][0]
        second = itemgraph[i][1]
        adamweight = adamvalues[i]
        # 此处建立无向图 子图规则化
        if first not in graph:
            graph[first] = []
        graph[first].append((second, adamweight))
        if second not in graph:
            graph[second] = []
        graph[second].append((first, adamweight))

    # for item in graph:
    #     graph[item] = graph[item].sort(key=takeSecond, reverse=True)
    #     if n_sample >= len(graph[item]):
    #         pass
    #     else:
    #         graph[item] = graph[item][: n_sample]
    return graph
graph = construct_graph(itemgraph=itemgraph, adamvalues=adamvalues, n_sample=3)
np.save('/content/graph.npy', graph)
graph

constructing itemgraph ...
 99.86 %

In [0]:
def constrcut_adj(args, graph, item_number):
    # 要在这实现子图规则化（就是实现
    print('constructing adjacency matrix ...')
    # 矩阵的每一行都对应这子图规则化选出的三个adamic weight最大的三个物品

    # 待开发
    # 2019/11/16 21:52需要在此函数设计一个item 2 adjacency id的index

In [0]:
# 2019/11/16 模型搭建与子图规则化
class TSCN(object):
    def __init__(self, args, n_item, adj_itemgraph):
        self._parse_args(args, adj_itemgraph)
        self._build_input()
        self._build_model(n_items)
        self._build_train()

    @staticmethod
    def get_initializer():
        return tf.contrib.layers.xavier_initializer()

    def _parse_args(self, args, adj_itemgraph):
        # 此处默认邻接矩阵的行数已经形成与itemid的查询表（index）
        self.adj_itemgraph = adj_itemgraph
        # 树的深度影响迭代次数
        self.k = args.k
        self.batch_size = args.batch_size
        self.n_rootnode = args.sample_size
        self.dim = args.dim
        self.l2_weight = args.l2_weight
        self.lr = self.lr
        if args.pooling == 'average':
            self.pooling_class = AveragePooling
        elif args.poolling == 'max':
            self.pooling_class = MaxPooling
        elif args.pooling == 'adamic':
            self.pooling_class = AdamicPooling
        else:
            raise Exception('Unknown pooling method: ' + args.pooling)
    
    def get_rootnode(self, seeds)

    def _build_input(self):
        self.item_indices = tf.placeholder(dtype=tf.int64, shape=[None], name='item_indices')
        self.labels = tf.placeholder(dtype=tf.float32, shape=[None], name='labels')

    def _build_model(self, n_item):
        self.


